In [3]:
import numpy as np
import pandas as pd
import time 
import datetime as datetime
import glob
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
np.random.seed(1)
import gym
import gym_banana
import argparse
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch.autograd import Variable

In [4]:
Pad = pd.read_csv('PadData_v2.csv')
Pad.head()

,currency pair,timestamp,bid provider,bid price,bid volume,ask provider,ask price,ask volume,pad
0,AUDUSD,02.01.2019 00:00:00,LP-3,0.72714,1000000.0,LP-2,0.72718,2000000.0,0.0
1,AUDUSD,02.01.2019 00:00:01,LP-2,0.72712,2000000.0,LP-2,0.72717,2000000.0,0.0
2,AUDUSD,02.01.2019 00:00:02,LP-1,0.72713,1000000.0,LP-2,0.72718,2000000.0,0.0
3,AUDUSD,02.01.2019 00:00:03,LP-2,0.72713,1000000.0,LP-2,0.72717,2000000.0,0.0
4,AUDUSD,02.01.2019 00:00:04,LP-2,0.72713,2000000.0,LP-2,0.72717,1000000.0,0.0


In [5]:
T = 3617
m = 10
to_draw = np.sort(Pad['timestamp'].unique())
ccy = np.sort(Pad['currency pair'].unique())
min_history = 500 # min episode length

    
def generate_episode(n,cur):
    _max = to_draw.shape[0]
    _end = min(n+T, _max)
    timeframe = to_draw[n:_end]
    other_bid = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    other_ask = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    i = 0
    for elem in ccy:
        tmp = Pad[Pad['currency pair'] == elem]
        if elem == cur:
            target_bid = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            target_ask = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
        else:
            other_bid[:,i] = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            other_ask[:,i] = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
            i += 1
    return target_bid, target_ask, other_bid, other_ask

def features(price_path,m):
    features = np.zeros((price_path.shape[0]-m,m))
    for i in range(m):
        features[:,i] = (np.log(price_path) - np.log(np.roll(price_path, i+1)))[m:]
    return features

def get_features(target_bid, target_ask, other_bid, other_ask, m):
    feature_span = features(target_bid,m)
    feature_span = np.append(feature_span, features(target_ask,m), axis = 1)
    for i in range(other_bid.shape[1]):
        feature_span = np.append(feature_span, features(other_bid[:,i],m), axis = 1)
    for j in range(other_ask.shape[1]):
        feature_span = np.append(feature_span, features(other_ask[:,j],m), axis = 1)
    return feature_span

def draw_episode(m, cur, min_history):
    '''
    Input:
        m, number of lag returns z_1,...z_m
        cur, currency pair that we target to trade
        min_history, min length of a valid episode
    '''
#     n = np.random.randint(to_draw.shape[0] - min_history)
    n = 1
    target_bid, target_ask, other_bid, other_ask = generate_episode(n,cur)
    feature_span = get_features(target_bid, target_ask, other_bid, other_ask, m)
    return target_bid, target_ask, feature_span

def calculate_reward(previous_action, current_action):
    pass

def draw_episode_with_episode_num(m, cur, min_history, n = None):
    '''
    Input:
        m, number of lag returns z_1,...z_m
        cur, currency pair that we target to trade
        min_history, min length of a valid episode
    '''
    if n is None:
        n = np.random.randint(to_draw.shape[0] - min_history)
    
    target_bid, target_ask, other_bid, other_ask = generate_episode(n,cur)
    feature_span = get_features(target_bid, target_ask, other_bid, other_ask, m)
    return target_bid, target_ask, feature_span
    
def test_policy(policy = None, episode=None, totalNumEpisodes=1):
    if episode is None:
        for i in range(totalNumEpisodes):
            ask = np.zeros((1, 1))
            bid = np.zeros((1,1 ))
            previous_action = 0
            reward_for_this_episode = 0
            while ask.shape[0] <= 3600 and bid.shape[0]<=3600:
                target_bid, target_ask, feature_span = draw_episode_with_episode_num(16, 'AUDUSD', 1000, 1)
                bid, ask, features = target_bid[16:], target_ask[16:], feature_span
            for t in range(3600):  # Don't infinite loop while learning
                reward_for_this_episode += max(max(((ask[t+1] - ask[t]) + (bid[t+1] - bid[t]))/2, -((ask[t+1] - ask[t]) + (bid[t+1] - bid[t]))/2), 0)
            print("The reward for episode No.{} is {}".format(1, reward_for_this_episode))
            
        # a part of code to figure out the maximum value we can get
                
    else:
        #run policy through episode
        #compute
        pass
    
def main():
    test_policy()

if __name__ == '__main__':
    main()

The reward for episode No.1 is 0.04295500000004049


In [ ]:


T = 3617
m = 10
to_draw = np.sort(Pad['timestamp'].unique())
ccy = np.sort(Pad['currency pair'].unique())
min_history = 500 # min episode length

    
def generate_episode(n,cur):
    _max = to_draw.shape[0]
    _end = min(n+T, _max)
    timeframe = to_draw[n:_end]
    other_bid = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    other_ask = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    i = 0
    for elem in ccy:
        tmp = Pad[Pad['currency pair'] == elem]
        if elem == cur:
            target_bid = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            target_ask = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
        else:
            other_bid[:,i] = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            other_ask[:,i] = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
            i += 1
    return target_bid, target_ask, other_bid, other_ask

def features(price_path,m):
    features = np.zeros((price_path.shape[0]-m,m))
    for i in range(m):
        features[:,i] = (np.log(price_path) - np.log(np.roll(price_path, i+1)))[m:]
    return features

def get_features(target_bid, target_ask, other_bid, other_ask, m):
    feature_span = features(target_bid,m)
    feature_span = np.append(feature_span, features(target_ask,m), axis = 1)
    for i in range(other_bid.shape[1]):
        feature_span = np.append(feature_span, features(other_bid[:,i],m), axis = 1)
    for j in range(other_ask.shape[1]):
        feature_span = np.append(feature_span, features(other_ask[:,j],m), axis = 1)
    return feature_span

def draw_episode(m, cur, min_history):
    '''
    Input:
        m, number of lag returns z_1,...z_m
        cur, currency pair that we target to trade
        min_history, min length of a valid episode
    '''
#     n = np.random.randint(to_draw.shape[0] - min_history)
    n = 1
    target_bid, target_ask, other_bid, other_ask = generate_episode(n,cur)
    feature_span = get_features(target_bid, target_ask, other_bid, other_ask, m)
    return target_bid, target_ask, feature_span
import gym
import gym_banana
import argparse
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch.autograd import Variable


#parser = argparse.ArgumentParser(description='PyTorch REINFORCE example')
# parser.add_argument('--gamma', type=float, default=0.99, metavar='G',
#                     help='discount factor (default: 0.99)')
# parser.add_argument('--seed', type=int, default=543, metavar='N',
#                     help='random seed (default: 543)')
# parser.add_argument('--render', action='store_true',
#                     help='render the environment')
# parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                     help='interval between training status logs (default: 10)')
#args = parser.parse_args()


env = gym.make('Banana-v0')
env.seed(1)
torch.manual_seed(1)


class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        #self.affine1 = nn.Linear(256, 3)
        self.A = Variable(torch.randn(256, 3), requires_grad=True)
        self.b = Variable(torch.randn(1), requires_grad=True)
        #torch.nn.init.xavier_uniform(self.affine1.weight)
        #print('The weight of the affine layer is', self.affine1.weight)
#         self.dropout = nn.Dropout(p=0)
 #       self.mu = Variable(torch.randn(3, 3), requires_grad=True)
        #self.mu = nn.Linear(1,3, bias = True)
                #torch.nn.init.xavier_uniform(self.affine1.weight)
        #print('The weight of the affine layer is', self.affine1.weight)
        # self.tanh = nn.Tanh()
        self.softmax = nn.Softmax()

        self.saved_log_probs = []
        self.rewards = []
        self.actions = []

    def forward(self, x, y):
        x = torch.matmul(x, self.A) 
#         x = self.dropout(x)
#         print(x.size())
        #mu_sigma = torch.matmul(y, self.mu)
#         print(mu_sigma.size())
        # action = self.tanh(x + mu_sigma)
        # action = self.softmax(action)
        action = self.softmax(x)
#         action_scores = self.affine2(x)
#         return F.softmax(action_scores, dim=1)
        return action


policy = Policy()

optimizer = optim.SGD([policy.A, policy.b], lr=1e-1)
eps = np.finfo(np.float32).eps.item()


def select_action(state, previous_action):
    state = torch.from_numpy(state).float()
    previous_action = torch.from_numpy(previous_action).float()
    probs = policy(state, previous_action)
    #return probs
    #m = Categorical(probs)
    #action = m.sample()
    #policy.saved_log_probs.append(m.log_prob(action))
    return probs.detach().numpy()
    #return action.item() - 1


def finish_episode(num):
    R = 0
    policy_loss = []
    returns = []
    for r in policy.rewards[::-1]:
        #print("reward",r * 1000000)
#         R = r
        returns.insert(0, R)
    returns = torch.tensor(returns)
    optimizer.zero_grad()
    # returns = (returns - returns.mean()) / (returns.std() + eps)
    for R in returns:
#         print(Variable(-torch.ones(log_prob.shape), requires_grad = True) * R)
        #print("return", R)
        #print("policy_append", Variable(-torch.ones(log_prob.shape) * 1000000, requires_grad = True) * R)
        #policy_loss.append( torch.tensor(-1000000* R))
        policy_loss.append(Variable(-torch.ones((1,)) * 1000000, requires_grad = True) * R)
 
    policy_loss = torch.cat(policy_loss).sum() 
    #policy_loss = policy.A.sum()
    policy_loss.backward()
    optimizer.step()
    print(policy.A.grad)
    #print(policy.b.grad)
    #print(list(policy.parameters()))
    print ("Finished {} episode and the policy loss is {}".format(num, policy_loss))
    policy.rewards = []
    #optimizer.zero_grad()
    #del policy.saved|_log_probs[:]


def main():
    for i_episode in range(100):
        ask = np.zeros((1, 1))
        bid = np.zeros((1,1 ))
        previous_action = np.array([0, 0, 1])
        while ask.shape[0] <= 3600 and bid.shape[0]<=3600:
            target_bid, target_ask, feature_span = draw_episode(16, 'AUDUSD', 1000)
            bid, ask, features = target_bid[16:], target_ask[16:], feature_span
        for t in range(3600):  # Don't infinite loop while learning
            state = feature_span[t]
            action = select_action(state, previous_action)
            reward = ((action) * ((ask[t+1] - ask[t]) + (bid[t+1] - bid[t]))/2).sum()
            #reward -= 0.01 * abs(previous_action - action)  
            # the current price is 
#             state, reward, done, _ = env.step(action) #change banana
  #          if args.render:
  #              env.render()
            #print('reward is', reward)
            policy.rewards.append(reward)
#             ep_reward += reward
            #policy.actions.append(action)
            previous_action = action


#         running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
        finish_episode(i_episode)
#         if i_episode % 5 == 0:
#             print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
#                   i_episode, ep_reward, running_reward))
#         if running_reward > env.spec.reward_threshold:
#             print("Solved! Running reward is now {} and "
#                   "the last episode runs to {} time steps!".format(running_reward, t))
#             break


if __name__ == '__main__':
    main()

2019-04-27 17:25:47,813 - root - INFO - BananaEnv - Version 0.1.0


/Users/yilun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:114: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


None
Finished 0 episode and the policy loss is 0.0
None
Finished 1 episode and the policy loss is 0.0
None
Finished 2 episode and the policy loss is 0.0
None
Finished 3 episode and the policy loss is 0.0
None
Finished 4 episode and the policy loss is 0.0
None
Finished 5 episode and the policy loss is 0.0
None
Finished 6 episode and the policy loss is 0.0
None
Finished 7 episode and the policy loss is 0.0
None
Finished 8 episode and the policy loss is 0.0
None
Finished 9 episode and the policy loss is 0.0
None
Finished 10 episode and the policy loss is 0.0
None
Finished 11 episode and the policy loss is 0.0
None
Finished 12 episode and the policy loss is 0.0
None
Finished 13 episode and the policy loss is 0.0
None
Finished 14 episode and the policy loss is 0.0
None
Finished 15 episode and the policy loss is 0.0
None
Finished 16 episode and the policy loss is 0.0
None
Finished 17 episode and the policy loss is 0.0
None
Finished 18 episode and the policy loss is 0.0
None
Finished 19 episo

In [28]:
for param in policy.parameters():
    print((param.grad))

None
None
None
None


In [30]:
print(list(policy.parameters()))

[Parameter containing:
tensor([[ 0.0322, -0.0276, -0.0121,  0.0293, -0.0588,  0.0375, -0.0129,  0.0318,
          0.0087, -0.0077,  0.0173,  0.0031,  0.0228, -0.0244, -0.0046, -0.0056,
          0.0091, -0.0002,  0.0546,  0.0194, -0.0233, -0.0377, -0.0105, -0.0270,
         -0.0200,  0.0030,  0.0373,  0.0340, -0.0611,  0.0387,  0.0175,  0.0593,
          0.0413, -0.0569, -0.0594, -0.0301,  0.0549, -0.0104,  0.0267, -0.0290,
          0.0613, -0.0264,  0.0469,  0.0007, -0.0329,  0.0321, -0.0332,  0.0184,
         -0.0180, -0.0069, -0.0601, -0.0298,  0.0339, -0.0152,  0.0623,  0.0501,
         -0.0029, -0.0417,  0.0381,  0.0194, -0.0404,  0.0406,  0.0379,  0.0554,
         -0.0350, -0.0103, -0.0012,  0.0091, -0.0474, -0.0444,  0.0340, -0.0147,
          0.0305,  0.0036,  0.0205,  0.0137,  0.0227,  0.0310, -0.0579,  0.0315,
         -0.0439, -0.0472,  0.0038, -0.0107,  0.0367, -0.0362, -0.0556,  0.0455,
         -0.0093,  0.0352,  0.0201, -0.0469,  0.0126,  0.0150, -0.0418, -0.0297,
     